# Estimating FLOPs requirements
**WORK IN PROGRESS**

In [1]:
import contextlib
import os, sys
sys.path.insert(0, os.path.join(os.path.dirname(os.path.abspath('')), 'python'))
from nns.nns import ModelSummary, reset_keras, printable_dataframe, estimate
from nns.model import Model
from tensorflow.python.keras import models
from tensorflow.python.keras.layers import Conv2D, Conv2DTranspose, LSTM, Activation
from tensorflow.python.keras.layers import Bidirectional, SimpleRNN, Dropout, Dense

from IPython.display import SVG
from tensorflow.python.keras.utils.vis_utils import model_to_dot

In [2]:
inference = []
training = []

# LSTM Anomaly Detect
Example code for neural-network-based anomaly detection of time-series data (uses LSTM). Super simple and not particularly usefull ([repo](https://github.com/aurotripathy/lstm-anomaly-detect)).  
1. Sequence length = 100
2. Number of features = 1
3. Batch size = 50
4. Data is synthetic.

Three LSTM layers (64, 256, 100). Input length is 100.

In [3]:
class Model01(Model):
    """ 
    Super simple and not particularly usefull:
        https://github.com/aurotripathy/lstm-anomaly-detect
    """
    def __init__(self):
        super().__init__('Model01: LSTM anomaly detection')

    def create(self):
        return models.Sequential([
            Model.Input((100,1)),
            LSTM(64, return_sequences=True, name='lstm1'),
            Dropout(0.2),
            LSTM(256, return_sequences=True, name='lstm2'),
            Dropout(0.2),
            LSTM(100, return_sequences=False, name='lstm3'),
            Dropout(0.2),
            Dense(1, activation='linear', name='output')
        ], name=self.name)

In [4]:
estimate(Model01(), inference, training)

W0522 16:24:30.435842 140259972081408 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.LSTM object at 0x7f9036e493c8>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.
W0522 16:24:30.444176 140259972081408 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.LSTM object at 0x7f90311def98>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.
W0522 16:24:30.447798 140259972081408 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.LSTM object at 0x7f903119d6d8>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.


,name,out_shape,gFLOPs,num_params,num_activations,params_mem (MB),activations_mem (MB)
0,input,"(100, 1)",0.000000e+00,0,100,0.000000,0.000400
1,lstm1 (LSTM),"(100, 64)",1.664000e-03,16896,89600,0.067584,0.358400
2,dropout,"(100, 64)",0.000000e+00,0,6400,0.000000,0.025600
3,lstm2 (LSTM),"(100, 256)",3.276800e-02,328704,358400,1.314816,1.433600
4,dropout_1,"(100, 256)",0.000000e+00,0,25600,0.000000,0.102400
5,lstm3 (LSTM),"(100,)",1.424000e-02,142800,140000,0.571200,0.560000
6,dropout_2,"(100,)",0.000000e+00,0,100,0.000000,0.000400
7,output,"(1,)",1.000000e-07,101,1,0.000404,0.000004
8,TOTAL,"(1,)",4.867210e-02,488501,620201,1.954004,2.480804


# Keras Anomaly Detection
A repository with various models (FCNN, LSTM and Conv) for detecting anomalies ([repo](https://github.com/chen0040/keras-anomaly-detection)).
 LSTM models are defined [here](https://github.com/chen0040/keras-anomaly-detection/blob/master/keras_anomaly_detection/library/recurrent.py)

In [5]:
class Model02(Model):
    """ 
    https://github.com/chen0040/keras-anomaly-detection/blob/master/keras_anomaly_detection/library/recurrent.py#L7
    """
    def __init__(self):
        super().__init__('Model02: LSTM anomaly detection')

    def create(self):
        # Sequence length = 210, number of features = 1, batch size = ?
        return models.Sequential([
            Model.Input((210,1)),
            LSTM(128, return_sequences=False, name='lstm1'),
            Dense(128, activation='linear', name='output')
        ], name=self.name)

In [6]:
estimate(Model02(), inference, training)

W0522 16:24:37.126768 140259972081408 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.LSTM object at 0x7f8ff85ce0f0>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.


,name,out_shape,gFLOPs,num_params,num_activations,params_mem (MB),activations_mem (MB)
0,input,"(210, 1)",0.000000,0,210,0.000000,0.000840
1,lstm1 (LSTM),"(128,)",0.013870,66560,376320,0.266240,1.505280
2,output,"(128,)",0.000016,16512,128,0.066048,0.000512
3,TOTAL,"(128,)",0.013886,83072,376658,0.332288,1.506632


In [7]:
class Model03(Model):
    """ 
    https://github.com/chen0040/keras-anomaly-detection/blob/master/keras_anomaly_detection/library/recurrent.py#L218
    """
    def __init__(self):
        super().__init__('Model03: bidirectional LSTM anomaly detection')

    def create(self):
        # Sequence length = 210, number of features = 1, batch size = ?
        return models.Sequential([
            Model.Input(shape=(210,1)),
            Bidirectional(LSTM(128, return_sequences=False), name='bLSTM'),
            Dense(128, activation='linear', name='output')
        ], name=self.name)

In [8]:
estimate(Model03(), inference, training)

W0522 16:24:40.003205 140259972081408 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.LSTM object at 0x7f8ff8567080>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.
W0522 16:24:40.007340 140259972081408 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.LSTM object at 0x7f8ff82b62b0>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.


,name,out_shape,gFLOPs,num_params,num_activations,params_mem (MB),activations_mem (MB)
0,input,"(210, 1)",0.000000,0,210,0.000000,0.000840
1,bLSTM (LSTM),"(256,)",0.027740,133120,752896,0.532480,3.011584
2,output,"(128,)",0.000033,32896,128,0.131584,0.000512
3,TOTAL,"(128,)",0.027773,166016,753234,0.664064,3.012936


# [MTSAnomalyDetection](https://github.com/jsonbruce/MTSAnomalyDetection)
 
Multidimensional Time Series Anomaly Detection (MTSAD). Model is implemented [here](https://github.com/jsonbruce/MTSAnomalyDetection/blob/master/ensemblation/model.py).

In [9]:
# In progress

In [ ]:
# estimate(Model04(), inference, training)

### Summary: inference

In [9]:
printable_dataframe(inference)

,Model,Input shape,#Parameters,Model size (MB) FP32,GFLOPs (multiply-add),Activation size (MB) FP32
0,Model01: LSTM anomaly detection,"(100, 1)",488501,1.954004,0.048672,2.480804
1,Model02: LSTM anomaly detection,"(210, 1)",83072,0.332288,0.013886,1.506632
2,Model03: bidirectional LSTM anomaly detection,"(210, 1)",166016,0.664064,0.027773,3.012936


### Summary: training

In [10]:
printable_dataframe(training)

,Model,Input shape,#Parameters,Model size (MB) FP32,GFLOPs (multiply-add),Activation size (MB) FP32
0,Model01: LSTM anomaly detection,"(100, 1)",488501,1.954004,0.146016,4.961608
1,Model02: LSTM anomaly detection,"(210, 1)",83072,0.332288,0.041659,3.013264
2,Model03: bidirectional LSTM anomaly detection,"(210, 1)",166016,0.664064,0.083319,6.025872
